# 1.1 Flower Classification with TPUs


## Tensor Processing Units (TPUs)
TPUs是专门用于深度学习任务的硬件加速器。Tensorflow 2.1通过Keras高级和较低级别得API使用定制的训练循环的模型可支持TPUs

### 1.  Enable TPUs in Tensorflow and Keras

把Notebook上的“加速器”开关切换到“TPU v3-8”，就可在Tensorflow Keras中启用TPU训练:

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = tf.keras.Sequential( … ) # define your model normally
    model.compile( … )

# train model normally
model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

TPUs是连接到网络的加速器，必须首先在网络上使用TPUClusterResolver()定位它们。  
另外两行为固定格式，外加一个TPUStrategy。这个对象包含必要的分布式训练代码，这些代码将在拥有8个计算核心的TPUs上工作(参见下面的硬件部分)。

最后，通过在在tpu_strategy.scope内实例化模型来使用TPUStrategy。这将在TPU上创建模型。模型大小只受TPU RAM的限制，而不受运行Python代码的VM上可用内存的数量的限制。模型创建和模型训练使用通常的Keras api

## 2. Adjust the batch size and learning rate  
要在TPU上快速运行，要增加批量大小。经验法则是每个内核使用128个元素的批处理。(例如:8核TPU的批处理大小为128*8=1024) 在这个大小下，TPU的128x128硬件矩阵乘法器(参见下面的硬件部分)最有可能保持繁忙状态。
在批量大小为8的各个核上看到加速。在上面的例子中，批量大小是通过下面代码根据核数进行缩放的:

In [ ]:
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync

在单个TPU v3-8上运行TPUStrategy时，核心计数为8。这是Kaggle上提供的硬件。它可以在谷歌云上提供更大的配置，称为TPU豆荚。  
![](./img/TPU_batch_size.png)  
随着批处理规模的增大，TPUs将更快地处理训练数据。这只有在较大的训练批量产生更多的“训练工作”并使模型更快地达到所需的精度时才有用。这就是为什么经验法则也要求随着批量大小的增加学习率也要增加。可以从成比例增加开始，但可能需要进行额外的调优，以找到给定模型和加速器的最佳学习速率计划。

## 3.optimize your data pipeline for a fast accelerator 
### tf.data.Dataset and TFRecords  
由于TPUs非常快，许多移植到TPU上的模型最后都会遇到数据瓶颈。TPU处于闲置状态，等待每个训练阶段的大部分数据。TPUs只从GCS(谷歌云存储)中读取训练数据)。如果GCS能够连续地从多个文件并行传输数据，那么它可以维持相当大的吞吐量。以下几个最佳实践将优化吞吐量:

>**对于TPU培训，在GCS中以合理数量(10s到100s)的大文件(10s到100s MB)组织数据。**

如果文件太少，GCS将没有足够的流来获得最大的吞吐量。如果文件太多，访问每个文件就会浪费时间。  
TPU培训的数据通常通过适当数量的较大文件进行分片。通常的容器格式是TFRecords。通过以下代码可以从TFRecords文件加载数据集:

In [ ]:
# On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset
filenames = tf.io.gfile.glob("gs://flowers-public/tfrecords-jpeg-512x512/*.tfrec") # list files on GCS
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...) # TFRecord decoding here...

要启用来自多个TFRecord文件的并行流，请这样修改代码:

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

# On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset
filenames = tf.io.gfile.glob("gs://flowers-public/tfrecords-jpeg-512x512/*.tfrec") # list files on GCS
dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
dataset = dataset.with_options(ignore_order)
dataset = dataset.map(...) # TFRecord decoding here...

这里有两个设置:  
>**num_parallel_reads=AUTO**,指示API从多个文件中读取数据(如果可用)。它会自动计算出有多少。  
**experimental_deterministic = False**, 禁止执行顺序执行数据读取。因为我们需要将数据打乱，所以秩序是不重要的。通过此设置，API可以在任何TFRecord流进时立即使用它。

这些代码片段中省略了一些细节，因此请检查示例以获得完整的数据管道代码。在Keras和TensorFlow 2.1中，还可以将训练数据作为内存中的numpy数组发送到TPUs。这是可行的，但不是最有效的方法，虽然对于适合内存的数据集，它可能是OK的。

## 4.TPU hardware   
TPU v3-8板大约20英寸(50厘米)，是一个相当大的硬件。它拥有4个双核TPU芯片，共8个TPU核。 
![](./img/TPU_hardware.png)  
每个TPU核心都有一个传统的向量处理部分(VPU)以及能够处理128x128矩阵的专用矩阵乘法硬件。这是专门加速机器学习工作负载的部分。  
TPUs配备128GB的高速内存，允许更大的批次、更大的型号和更大的训练输入。在上面的示例中，您可以尝试使用数据集中提供的512x512像素的输入图像，并且可以看到TPU v3-8很容易地处理它们。

## TPU monitor  
![](./img/TPU_monitor.png)  
当您在Kaggle上运行一个TPU工作负载时，当单击TPU测量时，会出现一个性能监视器。  
MXU百分比表示TPU计算硬件的使用效率。高越好。  
“空闲时间”百分比度量TPU空闲等待数据的频率。应该优化数据管道，使其尽可能低。  
测量值大约每10秒刷新一次，并且仅在TPU运行计算时才出现。  

## 更多信息和教程 
一个包含更多信息、最佳实践和样本的TPU实践教程可以在这里找到:  
[Keras and modern convnets, on TPUs](Keras and modern convnets, on TPUs)

## 基于TPU的竞赛-Flower Classification with TPUs
很难理解我们的自然世界到底有多广阔和多样。
世界上有超过5000种哺乳动物，10000种鸟类，30000种鱼类，令人惊讶的是，还有超过40万种不同类型的花。
在这个竞赛中，你需要建立一个机器学习模型来识别图像数据集中的花卉类型(为了简单起见，我们坚持使用100多种类型)。